In [0]:
import dlt
from pyspark.sql.functions import *

#load members csv to Bronze
@dlt.table(
    comment="Raw member data loaded from CSV",
    table_properties={
        "quality": "bronze",
        "pipelines.autoOptimize.managed": "true"
    }
)
def bronze_members():
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("/Volumes/capstone_hospital/data/raw/members.csv")
    return df


#load diagnosis csv to Bronze
@dlt.table(
    comment="Raw diagnosis data loaded from CSV",
    table_properties={
        "quality": "bronze",
        "pipelines.autoOptimize.managed": "true"
    }
)
def bronze_diagnosis():
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("/Volumes/capstone_hospital/data/raw/diagnosis_ref.csv")
    return df


#load claim csv to Bronze
@dlt.table(
    comment="Raw claim data loaded from CSV",
    table_properties={
        "quality": "bronze",
        "pipelines.autoOptimize.managed": "true"
    }
)
def bronze_claim():
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load("/Volumes/capstone_hospital/data/raw/claims_batch.csv")
    return df



The Delta Live Tables (DLT) module is not supported on this cluster.
 You should either create a new pipeline or use an existing pipeline to run DLT code.

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4760633582222195>, line 1
----> 1 import dlt
      2 from pyspark.sql.functions import col
      4 @dlt.table(
      5   name="capstone_hospital.bronze_members",
      6   comment="Raw members data ingested to Bronze layer",
   (...)
     10 )
     11 def bronze_members():

File /databricks/python_shell/lib/dbruntime/autoreload/discoverability/autoreload_discoverability_hook.py:96, in AutoreloadDiscoverabilityHook._patched_import(self, name, *args, **kwargs)
     90 if not self._should_hint and (
     91     (module := sys.modules.get(absolute_name)) is not None and
     92     (fname := get_allowed_file_name_or_none(module)) is not None and
     93     (mtime := os.stat(fname).st_mtime) > self.last_mtime_by_modname.get(
     94         absolute_name, float("inf")) and not self._should_hint):
     95     self._should_hint 

In [0]:
# Load claims stream JSON to Bronze
@dlt.table(
    comment="Raw claims stream data loaded from JSON",
    table_properties={
        "quality": "bronze",
        "pipelines.autoOptimize.managed": "true"
    }
)
def bronze_claims_stream():
    return (
        spark.read.format("json")
        .load("/Volumes/capstone_hospital/data/raw/claims_stream.json"))

# Load providers JSON to Bronze
@dlt.table(
    comment="Raw providers stream data loaded from JSON",
    table_properties={
        "quality": "bronze",
        "pipelines.autoOptimize.managed": "true"
    }
)
def bronze_providers():
    return (
        spark.read.format("json")
        .load("/Volumes/capstone_hospital/data/raw/providers.json")
    )



In [0]:
# Silver view: member : cleaned data with quality checks
@dlt.table(
  name="silver_members",
  comment="Cleaned and deduplicated Silver sales data",
   table_properties={
        "quality": "silver",
        "pipelines.autoOptimize.managed": "true"
    }
)
def silver_members():
    df = dlt.read("bronze_members")
    return df.filter(col("memberID").isNotNull()).dropDuplicates(["memberID"])

# Silver view: diagnosis : cleaned data with quality checks
@dlt.table(
    name="silver_diagnosis",
    comment="Cleaned data with quality checks",
    table_properties={
        "quality": "silver",
        "pipelines.autoOptimize.managed": "true"
    }
)
def silver_diagnosis():
    return dlt.read("bronze_diagnosis").filter(col("code").isNotNull()).dropDuplicates(["code"])

# Silver view: diagnosis : cleaned data with quality checks
@dlt.table(
    name="silver_claim",
    comment="Cleaned data with quality checks",
    table_properties={
        "quality": "silver",
        "pipelines.autoOptimize.managed": "true"
    }
)
def silver_claim():
    return dlt.read("bronze_claim").filter(col("ClaimID").isNotNull()).dropDuplicates(["ClaimID"])    

 # Silver view: claims stream : cleaned data with quality checks
@dlt.table(
    name="silver_claims_stream",
    comment="Cleaned data with quality checks",
    table_properties={
        "quality": "silver",
        "pipelines.autoOptimize.managed": "true"
    }
)
def silver_claim():
    return dlt.read("bronze_claims_stream").filter(col("ClaimID").isNotNull()).dropDuplicates(["ClaimID"])    

 # Silver view: provider : cleaned data with quality checks
@dlt.table(
    name="silver_providers",
    comment="Cleaned data with quality checks",
    table_properties={
        "quality": "silver",
        "pipelines.autoOptimize.managed": "true"
    }
)
def silver_claim():
    return dlt.read("bronze_providers").filter(col("ProviderID").isNotNull()).dropDuplicates(["ProviderID"])    

In [0]:
@dlt.table(
    name="gold_fraud_detection",
    comment="Filtered and joined data for fraud detection"
)
def gold_fraud_detection():
    # Load the tables
    silver_claim = spark.read.table("capstone_hospital.default.silver_claim")
    silver_diagnosis = spark.read.table("capstone_hospital.default.silver_diagnosis")
    silver_members = spark.read.table("capstone_hospital.default.silver_members")

    # Explode ICD10Codes into multiple rows
    exploded_claims = silver_claim.withColumn("ICD10Code", explode(split(col("ICD10Codes"), ",")))

    # Filter rows where ClaimDate is more than 24 hours before ServiceDate
    filtered_claims = exploded_claims.filter(datediff(col("ClaimDate"), col("ServiceDate")) > 1)

    # Join the tables
    result = filtered_claims.join(silver_members, "MemberID") \
                            .join(silver_diagnosis, filtered_claims["ICD10Code"] == silver_diagnosis["code"])

    return result